<img align="center" width="12%" style="padding-right:10px;" src="../Images/Ccom.png">

# Integrated Seabed Mapping Systems <a href="https://teams.microsoft.com/l/channel/19%3af7b302e1f5b84615a207b9e579d62166%40thread.tacv2/Lab%2520D?groupId=81cb9921-c02d-4e3d-b4d7-9ef6985e82b1&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="../Images/help.png"  title="Ask questions on Teams" align="right" width="10%" ></a><br><br>  Lab D: Orient and Ray Trace Beam

___

<img align="center" width="80%" style="padding-right:10px;" src="../Images/ccom874_LAB_D_Oblique_Integration_2017.jpg">

This lab is designed for you now that you can navigate the Tx and Rc for each shot time as you did in lab A

We are just going to take one transmission and, for the case of a single receiver beam:
•A –work out where the beam vector was pointing (Azimuth, Depression).
•B –work out where the beam at that TWTT actually hit the seafloor (in a local level, but ship’s head relative coordinate system).
For this you need to do:
•the Cone-Cone intersection Calculation(see separate worked example)

And you need to perform a ray trace. 

In [22]:
%load_ext autoreload
%autoreload 2

import numpy as np
from numpy import pi, sin, cos, log, log10, exp, nan
import matplotlib.pyplot as plt
import scipy.ndimage
import gsw
import sys
import os
from pathlib import Path

# The classes and data needed are in the lab A mycode and Data folders respectively

lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

# Get the data path
abs_path = os.path.abspath(os.path.curdir)

# import of your own classes

from mycode.Motion import Motion
from mycode.SSP import SSP
from mycode.vessel import Vessel
from mycode.position import Position

# By default represent numbers in numpy arrays with four decimals
np.set_printoptions(formatter={'all':lambda x: ' '+str('%.8f'%x)})



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


___
## C.0 Initialization

<img align="center" width="80%" style="padding-right:10px;" src="../Images/Initialization Parameters.jpg">


Update the code cell below to enter the lever arms (Note that this is exactly as in Lab A)

In Lab A we saw how to navigate the transducers, for this lab we will also need to see how they are oriented in space at the time of transmission and reception, in particular we want to determine the orientation of the long axes of the transducers with respect to which the beam steering will take place. In this case we will ignore the MRU (Motion Reference Unit) to SRF (Ship Reference Frame) misalignment. Also, we will consider that the heave solution is provided for the location of the Reference Point (RP)

Add the variables `bias_angles_trans` and `bias_angles_rec` as `numpy` arrays to your Vessel class (in the `__init__` method). 

Finally add the variable `beam_n` which will hold the index of the beam assigned to you.

In [23]:
# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel

vessel = Vessel()

# The transmit transducer
vessel.lever_arm_trans = np.array([16.26, -1.75,   4.15]).reshape((3, 1))

# The receive transducer
vessel.lever_arm_rec = np.array([14.82, -2.01,   4.17]).reshape((3, 1))

#The Positioning System
vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))

# The Motion Reference Unit
vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))
vessel.wl = -2.59

vessel.metadata["name"]="USNS Henson"
vessel.metadata["owned_by"]="United States Navy"
vessel.metadata["operated_by"]="United States Navy"
vessel.metadata["pos_source"]="NavCom (C-Nav)"
vessel.metadata["sonar"]="Kongsberg EM710"
vessel.metadata["mru"]="Applanix POS/MV 320"
vessel.metadata["loa"]=100

beam_n = 391

Now add the roll, pitch and yaw biases in the same fashion as you added the offset parameters, i.e., the roll is associated to the x-axis, so should be the first, etc. Make sure to use angles in radians!

In [24]:
# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel


# The transmit transducer
vessel.bias_angles_rec = np.array([0.127*pi/180,...,...]).reshape((3, 1))

# The receive transducer
vessel.bias_angles_rec = ...


___
## 1 The Bottom Strike of One Beam of one Swath to Analyze

<img align="center" width="80%" style="padding-right:10px;" src="../Images/OneBeam.png">

This shows swath 4070 from this sample data set (the same data as Lab A). It was collected while the vessel was turning (although yaw stabilization was deliberately disabled). The system was operating in +/-60 deg sector using dual swath. This is just the first swath. The EM710 was operating in shallow mode using a 0.2ms pulse. (standard for < 100m of water).

Each of the students in the class will be assigned a single (different) beam in the outer sector. For that beam, all the geometry of Tx. And Rc. will be established and the beam vector estimated. That beam vector should be reported in Azimuth and Depression angle.

From the sonar depth a ray trace, using that vector and the TWTT will be undertaken. The beam azimuth used should be that with respect to the ships heading at transmit i.e. Azimuth, w.r.t north – heading at transmit.

Finally knowing the radial distance and depth of the bottom detection relative to the sonar, the resulting bottom detection will be reported with respect to the RP (in meters along track and meters across track) and with respect to the sea surface, reduced for water level, heave, induced heave and sonar. 

This deliverable can be directly compared to the Kongsberg real time calculated value which is provided.

### 1.0 Load the data

The first step that we will take is to load the appropriate data. We will use the same data that was used in lab A i.e., the files `Lab_A_GNSS.txt`, `Lab_A_MRU.txt`

___
## 1 Calculating the Horizontal Location of the *Virtual Array*

For this lab we will use a *virtual array* to represent the location of the combined transmit and receive arrays at the moment that the signal reflects of the bottom

The virtual array is constructed at the mid point between the Tx (Transmit transducer) location on transmit, and the Rx (Receive transducer) location on receive. At any single time, the two arrays are actually offset as they cannot be installed at the same place on the hull. Additionally, as these are two separate times, the translation of the entire vessel between those two times should strictly be accounted for.

In real time, as the final navigation solution is not yet available, the convention is to have the stored location of the bottom strike referenced in a coordinate system that is relative to a fixed point in the vessel as it was at a specific epoch (usually the transmit time). We use the RP (Reference Point) not the sonar for this purpose. That coordinate system is oriented in a local level plane that is aligned along and across the vessel. As an example of why this is practical,, when editing swath data, one normally views the solutions in across track coordinates, that way one can see the shape of successive profiles and perform editing based on their coherence. Many of the artifacts present in swath data are oriented across the swath (e.g. refraction biases, wobbles).

For this lab, you will locate the Rx and Tx relative to the RP at that instant (using lever arms and instantaneous orientation). To account for the along track movement of the RP, the vessel speed and azimuth at that time can be used (and knowing the twtt~ 0.23 seconds). Additionally there will be a small azimuth rotation of the SRF between the two epochs.

___
### 1.0 Load the Auxilliary Data

The first step that we will take is to load the appropriate data. We will use the same data that was used in the other labs A i.e., instantiate the objects `pos`, `motions`, and `sound_speed_profile` in the same manner as in labs A and B using the files `Lab_A_GNSS.txt`, `Lab_A_MRU.txt`, and `Lab_A_SVP.txt` respectively. Make sure to have a Cartesian representation of the coordinates, use a UTM projection for this purpose.

In [25]:
# positions
pos = Position()
pos.read_jhc_file(str(lab_a)+'/Data/Lab_A_GNSS.txt')
# make sure that there is Cartesian representation of the positions
pos.carto_project('utm','ortho')

# Motion data
motions = ...
motions.read_jhc_file...

# Sound speed data
sound_speed_profile = SSP()
sound_speed_profile...

Opening GNSS data file:../Lab_A/Data/Lab_A_GNSS.txt
Opening motion data file:../Lab_A/Data/Lab_A_MRU.txt
Opening sound speed profile data file:../Lab_A/Data/Lab_A_SVP.txt


___
### 1.1 Load the Multi Beam EchoSounder (MBES ) Data




#### Details on the specific multi-sector swath to be analyzed:

The multibeam installed on the USNS Henson is an EM710 which operates using three sectors per swath and dual swath.

#### Sector-Specific Transmit Steering

We will examine the first swath of a dual swath cycle. Each of the three sectors can have different steering. For this particular case, where the yaw stabilization is turned off, the outermost sectors are steered with the same magnitude.

<img align="center" width="80%" style="padding-right:10px;" src="../Images/ping_data.jpg">

A single file `data_ping_4170.txt` is supplied which, together with the orientation and offsets, will allow you to do the full calculation.

You may ask what information do you have on the transmit sector for the beam you chose?

From the data telegram, there is lots of other information that is provided about each sector which we will read. However, the only things you will eventually need are:

- the transmit steering angle 
- the time delay between the first sector firing and the one you have chosen.

You will have been assigned one of the beams from the starboard sector (sector 2). Thus your actual time of transmission is the nominal time of the swath PLUS the extra delay while it waits for the other sectors to fire. For example, Sector 1 fired first (at the specified time). Sector 0 fires 0.4ms later and Sector 2 fires another 0.4ms after that (0.8ms w.r.t the central sector).

<img align="center" width="80%" style="padding-right:10px;" src="../Images/sector.jpg">

For your chosen sector, the two values you have to know are the transmit steering angle and the delay in firing that sector compared to the time the first sector fired.

The other information should now be familiar to all based on the integrated seabed mapping systems course discussions of multi-sector operation and working in the near field.

We will determine all needed values in the course of this lab.

<img align="center" width="80%" style="padding-right:10px;" src="../Images/ping_info.jpg">

Note that the final answer that you are working towards is contained in the file. In the file provided there are a list of ALL the 400 beams for a single swath. Starting at beam 0 (outermost on the port side) you can see the raw measurements (TWTT, Transmit steering, Receive steering and the transmit delay).

For each you will also see the “answer” – the official Kongsberg real-time calculation of the beam bottom strike. This is reported in a horizontal coordinate system centred on the RP and oriented along the ships heading at the time of transmit. Thus one has along track values that are typically 15-16m as the sonars are 15-16m ahead of the RP, and the across track values range from -160 m (to port side) to +156 m (to starboard side).

The resulting depths are not, however, from the RP; rather they are reported from the wave-free water surface. They are thus a combination of the depth below the transducer, the depth of the transducer (WLZ, heave, induced heave and Tx-Z in the SRF). You will note that they are ~ 97m with almost no variation acrossthe swath (a very-very flat seafloor).

In the steps of this lab you will be assigned a beam from the outer starboard sector, and at the end will estimate the bottom detection location in this coordinate system. ***You are unlikely to get exactly the same answer (we are using slightly different assumptions than Kongsberg) but you should try to be within ~ 0.1m in all axes (0.1% of Z).***


### 1.1.1 Create a Ping Class holding the 'Ping' Specific Data

Here we will load the data provided by the MBES. We do not yet have a class and associated parser for this. Start by defining a `Ping` class in the file `ping.py` that you will create in the mycode directory for Lab_A. Add member variables to hold all the information contained within the file.

The data contained in the file contains three sections, the first being general information for the ping - we will store them in the class objects in the following fashion:

The elements of the first section are general to all the data for this ping
    
    - lat               - scalar holding the latitude in radians
    - lon               - scalar holding the longitude in radians
    - tx_roll           - scalar holding the tx roll in radians
    - tx_pitch          - scalar holding the tx pitch in radians
    - tx_heading        - scalar holding the tx heading in radians
    - tx_heave          - scalar holding the tx heave in radians
    - tx_time           - datetime object holding the time 

The elements of the second section describe the parameters for each transmit sector for this ping (really a combination of three pings in this case)

    - tilt_angle        - numpy array holding the tilt angle for the transducer in radians
    - focus_range       - numpy array holding the distance from the Tx trasnducer to the focal point in meters
    - signal_length     - numpy array holding the duration of signal in seconds
    - tx_time_offset    - numpy array holding the time offset of transmit from tx_time for each sector in seconds
    - center_frequency  - numpy array holding the central frequency for each sector in Hz
    - mean_absorp_coeff - numpy array holding the Absorption coefficient for each sector dB/km
    - waveform_id       - numpy array holding thethe shape of the transmit pulse for each sector (integer)
    - sector_id         - numpy array identifying each sector (integer)
    
The elements of the third section describe the properties associated to the bottom detect of each beam, Note that much of this info is repeated from the previous sections or derived from other available parameters. We will parse this data anyway, even though this is dangerous as the data may be inconsistent.

    - beam              - numpy array holding the beam numbers (integer)
    - across            - numpy array holding the across track distance to the bottom for each beam in meters
    - along             - numpy array holding the along track distance to the bottom for each beam in meters
    - depth             - numpy array holding the depth to the bottom for each beam in meters
    - samp_nr           - numpy array holding the sample nr at which the bottom was found for each beam (integer) 
    - twtt              - numpy array holding the TWTT for each beam in seconds
    - range             - numpy array holding the slant range for each beam in meters
    - steer_rx          - numpy array holding the receive steering angle for each beam in radians
    - sector            - numpy array holding the sector (integer)
    - steer_tx          - numpy array holding the transmit steering angle for each beam in radians
    - tx_t_offset_beam  - numpy array holding the delay from tx_time as a timedelta object
    - ctr_freq_beam     - numpy array holding the central frequency for each beam in Hz

Just to be clear, the above is the variable names that you should define in the `__init__` method for the `Ping` class. 


___
### 1.1.2 Create a Read Method to Read the Ping Data

Add a method named `read` to the class `Ping` you created. The method should take as an argument the data path to the ping data file. The method should populate all the data fields defined in the `__init__` method. It is up to you to interpret the data in the file using the provided information, assign the right data to the right variables, and to ensure that the desired units are used in the class objects (you may have to cast some of the units)


In [26]:
ping = Ping()
ping.read('data_ping_4170.txt')

NameError: name 'Ping' is not defined

___
### 1.1.3 Sector Transmit Steering

We are examining the first swath of a dual swath cycle. Each of the three sectors can have different steering. For this particular case, when the yaw stabilization is turned off, the outermost sectors are steered with the same magnitude

#### 1.1.3.1 Explain why the central sector steering is different from the outer sectors?

In [ ]:
ans_1_1_3_1 = "Your answer here"

In [ ]:
#### 1.1.3.2 Explain why the central sector steering is different from the outer sectors?

In [ ]:
ans_1_1_3_2 = "Your answer here"

In [8]:
from datetime import datetime
a = datetime(none)

NameError: name 'none' is not defined